In [1]:
# Database Setup - Automatic Environment Detection
# This notebook works in BOTH Databricks and local Jupyter - NO CONFIGURATION NEEDED!

# Install required package (works in both environments)
%pip install loguru -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Restart Python (Databricks only - does nothing in local Jupyter)
try:
    dbutils.library.restartPython()
except NameError:
    print("✓ Local environment - no restart needed")

✓ Local environment - no restart needed


In [3]:
# CONFIGURATION - Edit these values as needed

# For Databricks: Set catalog and schema
# For Local: Set db_path (optional, defaults to "databases/granular_impact.db")

# Databricks widgets (if in Databricks, create widgets first)
try:
    dbutils.widgets.text("catalog", "onedata_us_east_1_shared_dit", "Catalog")
    dbutils.widgets.text("schema", "ssot_raw_ditagen_dit", "Schema")
    
    CATALOG = dbutils.widgets.get("catalog")
    SCHEMA = dbutils.widgets.get("schema")
    print(f"Databricks mode: {CATALOG}.{SCHEMA}")
except NameError:
    # Local environment - no widgets needed
    CATALOG = None
    SCHEMA = None
    DB_PATH = "databases/faq_update.db"  # Change if needed
    print(f"Local mode: {DB_PATH}")

Local mode: databases/faq_update.db


In [4]:
# SETUP DATABASE - Using Repository Pattern (Phase 5 Migration)

from pathlib import Path
from database.backends.factory import BackendFactory
from database.config import DatabaseConfig
from database.repository import ContentRepository, FAQRepository, AuditRepository
from database.schema_manager import SchemaManager
from utility.environment import is_databricks

# CONFIGURATION - Edit these to control behavior
DROP_EXISTING_TABLES = True  # Set to True to drop and recreate all tables
CREATE_TABLES = True          # Set to False to only connect (no table creation)
DRY_RUN = False              # Set to True to see what would happen without executing

# Schema directory configuration
SCHEMA_DIR = Path("database/sql/schema")  # Path to SQL schema files

print("=" * 70)
print("DATABASE SETUP - Repository Pattern")
print("=" * 70)
print(f"Drop existing tables: {DROP_EXISTING_TABLES}")
print(f"Create tables: {CREATE_TABLES}")
print(f"Dry run mode: {DRY_RUN}")
print(f"Schema directory: {SCHEMA_DIR}")
print()

# Step 1: Create database configuration
if is_databricks():
    config = DatabaseConfig(
        backend='databricks',
        catalog=CATALOG,
        schema=SCHEMA
    )
    print(f"📊 Databricks mode: {CATALOG}.{SCHEMA}")
else:
    config = DatabaseConfig(
        backend='sqlite',
        db_path=DB_PATH
    )
    print(f"💾 Local mode: {DB_PATH}")

print()

# Step 2: Create backend and schema manager
backend = BackendFactory.create_backend(config)
backend.connect()  # Must explicitly connect before using the backend
schema_manager = SchemaManager(backend, schema_dir=SCHEMA_DIR)

# Step 3: Drop existing tables if requested
# SchemaManager handles dependency order automatically via dependency_graph.yaml
if DROP_EXISTING_TABLES:
    if DRY_RUN:
        print("🔍 DRY RUN: Would drop existing tables")
        schema_manager.drop_schema(dry_run=True)
    else:
        print("🗑️  Dropping existing tables (in reverse dependency order)...")
        # Drop tables using SchemaManager (respects dependency graph)
        # Note: This will prompt for confirmation in interactive mode
        import sys
        import io
        from contextlib import redirect_stdout
        
        # Suppress confirmation prompt in notebook by mocking stdin
        old_input = __builtins__.input
        __builtins__.input = lambda _: "DELETE ALL DATA"
        
        try:
            schema_manager.drop_schema(cascade=False, dry_run=False)
        finally:
            __builtins__.input = old_input
    print()

# Step 4: Create tables if requested
if CREATE_TABLES and not DRY_RUN:
    print("🏗️  Creating tables...")
    try:
        schema_manager.create_schema()
        print("   ✓ All tables created successfully")
    except Exception as e:
        print(f"   ⚠ Error creating tables: {e}")
    print()

# Step 5: Initialize repositories
content_repo = ContentRepository(backend)
faq_repo = FAQRepository(backend)
audit_repo = AuditRepository(backend)

print("=" * 70)
if DRY_RUN:
    print("✅ DRY RUN COMPLETE - No changes made")
else:
    print("✅ DATABASE SETUP COMPLETE")
print("=" * 70)
print(f"Environment: {config.backend}")
print(f"Backend: {backend.__class__.__name__}")
if CREATE_TABLES and not DRY_RUN:
    print("Tables: Created (or already exist)")
else:
    print("Tables: Connection only (no table creation)")
print()
print("Available Repositories:")
print("  - content_repo: ContentRepository")
print("  - faq_repo: FAQRepository")
print("  - audit_repo: AuditRepository")
print()

2025-11-03 13:56:54.930 | INFO     | database.backends.factory:create_backend:133 - Creating backend: sqlite (sqlite://databases/faq_update.db)
2025-11-03 13:56:54.931 | DEBUG    | database.backends.sqlite.connection:__init__:90 - SQLiteConnectionManager initialized: db_path=databases/faq_update.db, timeout=30.0, journal_mode=WAL
2025-11-03 13:56:54.931 | INFO     | database.backends.sqlite.backend:__init__:128 - SQLiteBackend initialized: db_path=databases/faq_update.db
2025-11-03 13:56:54.931 | DEBUG    | database.backends.factory:create_backend:141 - Backend stored as singleton
2025-11-03 13:56:54.934 | DEBUG    | database.backends.sqlite.connection:get_connection:110 - Creating new SQLite connection to databases/faq_update.db
2025-11-03 13:56:54.936 | DEBUG    | database.backends.sqlite.connection:_apply_pragma_settings:131 - Foreign keys enabled
2025-11-03 13:56:54.938 | DEBUG    | database.backends.sqlite.connection:_apply_pragma_settings:138 - Journal mode set to: wal
2025-11-03

DATABASE SETUP - Repository Pattern
Drop existing tables: True
Create tables: True
Dry run mode: False
Schema directory: database\sql\schema

💾 Local mode: databases/faq_update.db

🗑️  Dropping existing tables (in reverse dependency order)...

🏗️  Creating tables...


2025-11-03 13:56:55.114 | DEBUG    | database.schema_manager:_execute_sqlite_statements:642 -   Executing statement 8/8
2025-11-03 13:56:55.116 | DEBUG    | database.backends.sqlite.backend:execute_command:321 - Command affected -1 rows
2025-11-03 13:56:55.119 | SUCCESS  | database.schema_manager:_execute_sql:578 - ✅ Completed: Creating table: content_change_log
2025-11-03 13:56:55.122 | DEBUG    | database.schema_manager:_load_sql_file:288 - Loading SQL file: database\sql\schema\tables\08_faq_impact_decisions.sql
2025-11-03 13:56:55.124 | INFO     | database.schema_manager:_execute_sql:561 - Executing: Creating table: faq_impact_decisions
2025-11-03 13:56:55.127 | DEBUG    | database.schema_manager:_execute_sqlite_statements:636 -   Executing 10 statements
2025-11-03 13:56:55.128 | DEBUG    | database.schema_manager:_execute_sqlite_statements:642 -   Executing statement 1/10
2025-11-03 13:56:55.129 | DEBUG    | database.backends.sqlite.backend:execute_command:321 - Command affected -1

   ✓ All tables created successfully

✅ DATABASE SETUP COMPLETE
Environment: sqlite
Backend: SQLiteBackend
Tables: Created (or already exist)

Available Repositories:
  - content_repo: ContentRepository
  - faq_repo: FAQRepository
  - audit_repo: AuditRepository



In [5]:
# (OPTIONAL) Verify the setup by listing tables using repository

print("📊 Verifying database setup...")
print()

try:
    # Get file stats from content repository
    file_stats = content_repo.get_file_stats()
    chunk_stats = content_repo.get_chunk_stats()
    
    print(f"✓ Content Repository:")
    print(f"  - Total files: {file_stats.get('total_files', 0)}")
    print(f"  - Unique files: {file_stats.get('unique_files', 0)}")
    print(f"  - Total chunks: {chunk_stats.get('total_chunks', 0)}")
    print(f"  - Unique checksums: {chunk_stats.get('unique_checksums', 0)}")
    
    # Get FAQ stats from FAQ repository
    try:
        faq_stats = faq_repo.get_faq_stats()
        print()
        print(f"✓ FAQ Repository:")
        print(f"  - Total questions: {faq_stats.get('total_questions', 0)}")
        print(f"  - Total answers: {faq_stats.get('total_answers', 0)}")
    except Exception as e:
        print(f"  ⚠ Could not get FAQ stats: {e}")
    
    # Get audit stats
    try:
        audit_stats = audit_repo.get_detection_run_stats()
        print()
        print(f"✓ Audit Repository:")
        print(f"  - Total change logs: {audit_stats.get('total_change_logs', 0)}")
        print(f"  - Total detection runs: {audit_stats.get('total_runs', 0)}")
    except Exception as e:
        print(f"  ⚠ Could not get audit stats: {e}")
    
    print()
    print("=" * 70)
    print("✅ DATABASE VERIFICATION COMPLETE")
    print("=" * 70)
    
    # Show table structure for reference
    if not is_databricks():
        # For SQLite, we can query sqlite_master
        try:
            tables_query = """
                SELECT name FROM sqlite_master 
                WHERE type='table' AND name NOT LIKE 'sqlite_%'
                ORDER BY name
            """
            results = backend.execute_query(tables_query)
            
            print()
            print(f"📋 Database Tables ({len(results)}):")
            for row in results:
                table_name = row['name']
                count_query = f"SELECT COUNT(*) as count FROM {table_name}"
                count_result = backend.execute_query(count_query)
                count = count_result[0]['count'] if count_result else 0
                print(f"  - {table_name}: {count} rows")
        except Exception as e:
            print(f"  ⚠ Could not list tables: {e}")
    
except Exception as e:
    print(f"❌ Error during verification: {e}")
    print()
    print("💡 This is normal if tables are newly created (empty).")
    print("   Try ingesting some data and re-run this cell.")

2025-11-03 13:56:55.174 | DEBUG    | database.repository.content_repository:get_file_stats:837 - Calculating file statistics
2025-11-03 13:56:55.175 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-03 13:56:55.176 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-03 13:56:55.177 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-03 13:56:55.178 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-03 13:56:55.179 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-03 13:56:55.180 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-03 13:56:55.180 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-03 13:56:55.181 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-03 

📊 Verifying database setup...

✓ Content Repository:
  - Total files: 0
  - Unique files: 0
  - Total chunks: 0
  - Unique checksums: 0

✓ FAQ Repository:
  - Total questions: 0
  - Total answers: 0
  ⚠ Could not get audit stats: 'AuditRepository' object has no attribute 'get_detection_run_stats'

✅ DATABASE VERIFICATION COMPLETE

📋 Database Tables (8):
  - content_change_log: 0 rows
  - content_chunks: 0 rows
  - content_repo: 0 rows
  - faq_answer_sources: 0 rows
  - faq_answers: 0 rows
  - faq_impact_decisions: 0 rows
  - faq_question_sources: 0 rows
  - faq_questions: 0 rows
